## 1) Setup environment

!pip -q install pandas 

## 2) Merge extra columns into column 2

In [11]:
import csv
import pandas as pd

rows = []
with open("../data/test/sieve.csv", "r", encoding="utf-8", errors="ignore") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        if len(row) > 2:
            # first cell = category, join everything else as log
            category = row[0]
            log = ",".join(row[1:]).strip()
            rows.append([category, log])
        elif len(row) == 2:
            rows.append(row)
        else:
            # empty or malformed line; keep placeholder for inspection
            rows.append([None, ",".join(row)])

df = pd.DataFrame(rows, columns=["category", "log"])
print(df.shape)
df.head()


(100000, 2)


,category,log
0,authentication-failed,[Thu Dec 17 02:47:06 1992] [error] [client 42....
1,authentication-failed,[Sun Mar 05 13:11:21 2017] [error] [client 15....
2,authentication-failed,[Tue Oct 06 14:38:18 1987] [error] [client 178...
3,authentication-failed,[Fri Mar 23 00:47:56 1979] [error] [client 91....
4,authentication-failed,[Fri Jun 03 16:48:40 1994] [error] [client 174...


## 3)Load and Inspect SIEVE dataset

In [9]:
import pandas as pd

csv_path = "../data/test/sieve.csv"
df = pd.read_csv(csv_path, on_bad_lines='skip')

print(df.shape)
df.head()

(81251, 2)


,category,log
0,authentication-failed,[Thu Dec 17 02:47:06 1992] [error] [client 42....
1,authentication-failed,[Sun Mar 05 13:11:21 2017] [error] [client 15....
2,authentication-failed,[Tue Oct 06 14:38:18 1987] [error] [client 178...
3,authentication-failed,[Fri Mar 23 00:47:56 1979] [error] [client 91....
4,authentication-failed,[Fri Jun 03 16:48:40 1994] [error] [client 174...


## 4) Pre-processing of Dataset

In [13]:
# Relabel columns
df = df.rename(columns={"category": "label", "log": "text"})

#Eliminate NaN rows or Empty rows
df = df.dropna(subset=["label", "text"])
df = df.drop_duplicates(subset=["label", "text"]).reset_index(drop=True)

#Remove whitespaces
df["label"] = df["label"].astype(str).str.strip()
df["text"] = df["text"].astype(str).str.strip()

print(df["label"].value_counts())
df.head()                               

label
authentication-failed           3334
authentication-success          3334
connection-failed               3334
connection-opened               3334
database-operation              3334
directory-changed               3334
directory-deleted               3334
directory-created               3334
file-action-failure             3334
file-deleted                    3333
file-read                       3333
file-modification               3333
file-write                      3333
hardware-monitoring             3333
process-info                    3333
http-request-failure            3333
http-request-success            3333
ids-alert                       3333
network-filtered                3333
network-traffic                 3333
process-ended                   3333
process-error                   3333
user-creation                   3333
process-shutdown                3333
process-started                 3333
system-configuration-changed    3333
user-logout                     

,label,text
0,authentication-failed,[Thu Dec 17 02:47:06 1992] [error] [client 42....
1,authentication-failed,[Sun Mar 05 13:11:21 2017] [error] [client 15....
2,authentication-failed,[Tue Oct 06 14:38:18 1987] [error] [client 178...
3,authentication-failed,[Fri Mar 23 00:47:56 1979] [error] [client 91....
4,authentication-failed,[Fri Jun 03 16:48:40 1994] [error] [client 174...


## 5) Stratified Split

In [17]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df["label"])
print("Train set size:", train_df.shape)
print("Validation set size:", val_df.shape)
print("Test set size:", test_df.shape)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Train set size: (79980, 2)
Validation set size: (9997, 2)
Test set size: (9998, 2)
Train: 79980, Val: 9997, Test: 9998


## 6) Check if data is adequate for training

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(lowercase=False, strip_accents=None, ngram_range=(1,2), min_df=1)),
    ("clf", LinearSVC())
])

pipeline.fit(train_df["text"], train_df["label"])
val_pred = pipeline.predict(val_df["text"])
test_pred = pipeline.predict(test_df["text"])

print("Validation Report:")
print(classification_report(val_df["label"], val_pred))
print(f"Macro-F1 (Val): {f1_score(val_df['label'], val_pred, average='macro'):.4f}")
print(f"Macro-F1 (Test): {f1_score(test_df['label'], test_pred, average='macro'):.4f}")


Validation Report:
                              precision    recall  f1-score   support

       authentication-failed       1.00      0.94      0.97       334
      authentication-success       0.95      1.00      0.97       334
           connection-closed       0.92      0.99      0.95       331
           connection-failed       0.95      0.99      0.97       333
           connection-opened       0.99      0.95      0.97       333
          database-operation       1.00      1.00      1.00       333
           directory-changed       0.97      0.73      0.84       333
           directory-created       1.00      1.00      1.00       334
           directory-deleted       0.83      0.93      0.88       333
         file-action-failure       1.00      1.00      1.00       334
                file-deleted       0.92      0.80      0.86       333
           file-modification       0.79      0.98      0.87       333
                   file-read       1.00      1.00      1.00       334


## 7) Generate JsonL files for finetuning

In [19]:
import json
from pathlib import Path

out_dir = Path("../data/training/classification/sieve_prepped")
out_dir.mkdir(exist_ok=True)

def write_jsonl(df, path, mode="instruction"):
    with open(path, "w", encoding="utf-8") as f:
        for _, r in df.iterrows():
            if mode == "instruction":
                obj = {
                    "instruction": "Classify the SIEM event type for this log line.",
                    "input": r["text"],
                    "output": r["label"]
                }
            else:
                obj = {"input": r["text"], "label": r["label"]}
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")

# Save both formats
write_jsonl(train_df, out_dir / "train.instruction.jsonl", "instruction")
write_jsonl(val_df,   out_dir / "val.instruction.jsonl", "instruction")
write_jsonl(test_df,  out_dir / "test.instruction.jsonl", "instruction")

write_jsonl(train_df, out_dir / "train.classification.jsonl", "classification")
write_jsonl(val_df,   out_dir / "val.classification.jsonl", "classification")
write_jsonl(test_df,  out_dir / "test.classification.jsonl", "classification")

print("Files saved to:", out_dir)


Files saved to: ..\data\training\classification\sieve_prepped


## 8) Verify formating of JsonL

In [ ]:
import json
pd.read_json(out_dir / "val.instruction.jsonl", lines=True).head()